<a href="https://colab.research.google.com/github/anshupandey/Microsoft_Azure_Implementing_Enterprise_GenAI_Applications_with_Azure_OpenAI-/blob/main/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Implementation with Azure OpenAI and LangChain

In [1]:
!pip install langchain-community langchainhub langchain-openai chromadb langchain langchain-experimental --quiet


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tavily-python 0.3.1 requires tiktoken==0.5.2, but you have tiktoken 0.7.0 which is incompatible.
steamship 2.17.34 requires tiktoken~=0.4.0, but you have tiktoken 0.7.0 which is incompatible.
llama-index 0.9.48 requires requests>=2.31.0, but you have requests 2.28.2 which is incompatible.
llama-index-legacy 0.9.48 requires requests>=2.31.0, but you have requests 2.28.2 which is incompatible.
llama-index-core 0.10.18.post1 requires requests>=2.31.0, but you have requests 2.28.2 which is incompatible.
langflow 0.6.10 requires pydantic<3.0.0,>=2.6.0, but you have pydantic 1.10.15 which is incompatible.
langflow 0.6.10 requires tiktoken<0.7.0,>=0.6.0, but you have tiktoken 0.7.0 which is incompatible.


In [2]:
!pip install jq pypdf --quiet

  error: subprocess-exited-with-error
  
  × Building wheel for jq (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_ext
      Executing: ./configure CFLAGS=-fPIC -pthread --disable-maintainer-mode --with-oniguruma=builtin
      error: [WinError 2] The system cannot find the file specified
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for jq
ERROR: Could not build wheels for jq, which is required to install pyproject.toml-based projects


In [3]:
api_key = "ed1a587475b74a178e4da9f49c27839e"
api_version = "2023-07-01-preview" # "2023-05-15"
azure_endpoint = "https://anshugpt.openai.azure.com/"
model_name = "gpt-35-turbo"
embedding_model = "text-embedding-ada-002"

In [4]:
import os
os.environ["OPENAI_API_VERSION"] = api_version
os.environ["AZURE_OPENAI_ENDPOINT"] = azure_endpoint
os.environ["AZURE_OPENAI_API_KEY"] = api_key

In [5]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [6]:
doc_paths = ["https://www.morningstar.com/content/dam/marketing/shared/research/methodology/771945_Morningstar_Rating_for_Funds_Methodology.pdf",
             "https://www.morningstar.in/docs/methodology/CategoryDefinitionsIndiaV3.pdf",
             "https://s21.q4cdn.com/198919461/files/doc_downloads/press_kits/2016/Morningstar-Sustainable-Investing-Handbook.pdf"]

In [7]:
from langchain_community.document_loaders import PyPDFLoader

loaders = [PyPDFLoader(pdf, extract_images=False) for pdf in doc_paths]

docs = []

for loader in loaders:
    doc = loader.load()
    docs.extend(doc)

In [8]:
len(docs)

46

In [9]:
docs = [doc for doc in docs if len(doc.page_content.strip())>100]
len(docs)

45

In [10]:
sum(len(doc.page_content) for doc in docs)/len(docs)

2665.911111111111

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3500, chunk_overlap=500)
splits = text_splitter.split_documents(docs)
len(splits)


53

In [12]:
print(splits[2].page_content)

©2021 Morningstar, Inc. All rights reserved. The information in this document is the property of Morningstar, Inc. Reproduction or transcription by any means, in whole or in part, without the prior written 
consent of Morningstar, Inc., is prohibited.
 The Morningstar RatingTM for Funds    August 2021 Page 3 of 21
3
3
3bond funds domiciled in Europe against other European high-yield bond funds. For more information 
about available categories, please contact your local Morningstar office.
Style Profiles
A style profile may be considered a summary of a fund’s risk-factor exposures. Fund categories 
define groups of funds whose members are similar enough in their risk-factor exposures that return 
comparisons between them are useful.
The risk factors on which fund categories are based can relate to value-growth orientation; 
capitalization; industry sector, geographic region, and country weights; duration and credit quality; 
historical return volatility; beta; and many other investment 

In [13]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(azure_deployment=embedding_model)

In [30]:
# initializng the vector store using CHromaDB
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceBgeEmbeddings
embedding_model_name = "BAAI/bge-large-en-v1.5"
embeddings = HuggingFaceBgeEmbeddings(  model_name=embedding_model_name,)

vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [17]:
retrieved_docs = retriever.invoke("What is Large Cap equity fund")
len(retrieved_docs)

6

In [18]:
print(retrieved_docs[0].page_content)

?  
 
 
 
 
 
 
 
 
 
 
 Categor y Definitions  
India 
Equity  
 
Large-Cap 
Large-Cap funds primarily consist of stocks which a re the Top 100 stocks by full market capitalization  of 
the equity market. These funds invest at least 80% of total assets in Indian equities and the balance can 
be invested in other asset classes such as fixed in come and overseas equities, among others. Funds in 
this category would invest at least 80% of their to tal assets in large-cap stocks. 
Morningstar Category Index: S&P BSE 100 TR 
 
Mid-Cap 
Mid-Cap funds primarily consist of stocks ranked 10 1st to 250th by full market capitalization of the 
equity market. These funds invest at least 65% of t otal assets in Indian equities, and the balance can  be 
invested in other asset classes such as fixed incom e and overseas equities, among others. Funds in thi s 
category would invest at least 65% of their total a ssets in mid-cap stocks. 
Morningstar Category Index: S&P BSE Mid Cap TR 
 
Small-Cap 
Smal

In [19]:
print(retrieved_docs[1].page_content)

©2019 Morningstar, Inc. All rights reserved. The in formation in this document is the property of Morni ngstar, Inc. Reproduction or transcription by any m eans, in whole or part, without  
the prior written consent of Morningstar, Inc., is prohibited. Category Definitions , India  | 26 February 2021  Page 2 of 12  
Multi- Cap 
Multi-Cap funds invest at least 75% of their total assets in Indian equities, and the balance can be 
invested in other asset classes such as fixed incom e and overseas equities, among others. These funds 
will invest a minimum of 25% each in Large Cap, Mid  Cap and Small Cap stocks. 
Morningstar Category Index: S&P BSE 500 TR 
 
 
Large & Mid- Cap 
Large & Mid-Cap funds primarily consist of stocks w hich are the Top 250 stocks by full market 
capitalization of the equity market. These funds in vest at least 70% of total assets in Indian equitie s and 
the balance can be invested in other asset classes such as fixed income and overseas equities, among 
others. F

In [20]:
print(retrieved_docs[2].page_content)

©2019 Morningstar, Inc. All rights reserved. The in formation in this document is the property of Morni ngstar, Inc. Reproduction or transcription by any m eans, in whole or part, without  
the prior written consent of Morningstar, Inc., is prohibited. Category Definitions , India  | 26 February 2021  Page 8 of 12  
Allocation 
 
Conservative Allocation 
Conservative Allocation funds invest across asset c lasses such as equity, fixed income and cash. These  
funds tend to make smaller allocations to equities than Balanced Allocation funds. The allocation to 
Indian equities ranges between 10-25% of total asse ts in the normal running of the fund. 
Morningstar Category Index: CRISIL Hybrid 85+15 - C onservative Index 
 
Balanced Allocation 
Balanced Allocation funds invest across asset class es such as equity, fixed income and cash. These 
funds tend to make larger allocations to equities t han Conservative Allocation funds. The allocation t o 
Indian equities usually ranges from 40%–60

### Creating retriever TOol

In [22]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "searchCapitalMarket",
    "Searches and returns excerpts from Morning start investment strategies and documents about trading stocks markets shares capital markets",
)
tools = [tool]

In [23]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [28]:
from langchain.agents import initialize_agent, load_tools, AgentType
from langchain_openai import AzureOpenAI
llm = AzureOpenAI(deployment_name=model_name)



agent = initialize_agent(tools , llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)


C:\Users\Anshu Pandey\AppData\Roaming\Python\Python310\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [29]:
result = agent.run({"input": "hi, i m bob"})

print(result)

C:\Users\Anshu Pandey\AppData\Roaming\Python\Python310\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 who is bob?
Action: searchCapitalMarket
Action Input: bob 
Observation: [Document(page_content='γ⎧\uf8f1γ>γ≠ ⎪\uf8f4=γ⎨\uf8f2\n⎪\uf8f4γ= ⎩\uf8f3-\xad‐W-\xad‐ -\xad‐1, 0u(W)\nln(W) 0\n( ) ( )\n( )1\nCE\nE ln 1 ERE 1 ER 1, 01 ER\ne0−−γγ\n⎡\uf8ee+⎤\uf8f9⎣\uf8f0⎦\uf8fb⎧\uf8f1⎡\uf8ee⎤\uf8f9+ γ>− γ≠ ⎪\uf8f4⎣\uf8f0⎦\uf8fb +=⎨\uf8f2\n⎪\uf8f4γ= ⎩\uf8f3γ⎧\uf8f1 + γ>γ≠+=⎨\uf8f2++ γ= ⎩\uf8f3-\xad‐\n0\n0\n0W u(1 TR) -\xad‐1, \t\r \xa0 0u(W(1 TR))\nln(W ) u(1 TR) 0\n denote the certainty equivalent geometric excess return for a given value of  \n [1] u(E[W])>E[u(W)]  \n \n \n[2]  \n \n \n[3]   \n \n \n \nγ \n \n \n[4]  \n  \n \n \n [5] 01W1 RF=+01W1 RF=+ \n \n \n [6]  \xa0u(W!(1+TR))=!!!"\n!!!"=u(1+ER)=−!!!"!!\n! \xa0 \xa0𝛾𝛾>−1,𝛾𝛾≠0\nln1+𝐸𝐸𝐸𝐸 \xa0 \xa0𝛾𝛾=0  \xa0\n \n \n [7] ER=ER=the geometric excess return =!!!"\n!!!!−1 \n \nCEER ( )γCEER ( )γ \n \nγ \n \n[8]   \n \n \n[9]   \n  \n \n \n \nCEERCEER  \n \n- γ \n \n  [10]() ( ) ( )CEu 1 ER E u 1 ER ⎡\uf8ee⎤\uf

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 8193 tokens, however you requested 14751 tokens (14495 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [ ]:
result = agent_executor.invoke({"input": "What is large cap market?"})
print(result)

{'input': 'What is large cap market?', 'output': "The large-cap market refers to the segment of the stock market that consists of companies with a large market capitalization. Market capitalization is calculated by multiplying the total number of a company's outstanding shares by its current stock price. Large-cap companies are typically well-established and have a market capitalization of billions of dollars. These companies are often considered to be more stable and less volatile compared to small-cap or mid-cap companies. Investors often look to large-cap stocks for their potential for steady growth and dividend payments."}
